In [8]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

X_train = np.genfromtxt('data/X_train.txt', delimiter=None) 
Y_train = np.genfromtxt('data/Y_train.txt', delimiter=None) 
X_test = np.genfromtxt('data/X_test.txt', delimiter=None) 

X_train, X_val, Y_train, Y_val = ml.splitData(X_train, Y_train, 0.80)

#random forrest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=0)

#knn classifier
knn_classifier = KNeighborsClassifier(metric_params=None, n_jobs=1, n_neighbors=5, p=1, weights='uniform')

#neural network 
nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

                    
#voting method
voting_classifier = VotingClassifier(estimators=[('rf', rf_classifier), ('knn', knn_classifier), ('nn', nn_clf)], voting='soft')

voting_classifier.fit(X_train, Y_train)

pred_soft = voting_classifier.predict_proba(X_train)
fpr, tpr, thresholds = roc_curve(Y_train, pred_soft[:,1])
roc_auc = auc(fpr, tpr)
print("Training data performance: " + str(roc_auc))


Training data performance: 0.980489347306162


In [9]:
pred_soft = voting_classifier.predict_proba(X_val)
fpr, tpr, thresholds = roc_curve(Y_val, pred_soft[:,1])
roc_auc = auc(fpr, tpr)
print("Validation data performance: " + str(roc_auc))

Validation data performance: 0.7417395094688723


In [11]:
pred_soft = voting_classifier.predict_proba(X_test)
np.savetxt('ensemble.txt',
np.vstack( (np.arange(len(pred_soft)) , pred_soft[:,1]) ).T,'%d, %.2f',header='ID,Prob1',comments='',delimiter=',');